In [ ]:
#importing libraries
import pandas as pd
from tensorflow.keras.optimizers import RMSprop,Adam
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras import Model
from keras import Model
from os import getcwd
from keras.layers import Dense, Activation, BatchNormalization
import tensorflow as tf

In [ ]:
tf.test.is_gpu_available(
    cuda_only=False, min_cuda_compute_capability=None
)

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau
earlyStopping = EarlyStopping(monitor='val_accuracy', patience = 15, verbose=0, mode='max')
mcp_save_xc = ModelCheckpoint(filepath='mdl_wts.hdf5', save_best_only=True, monitor='val_accuracy', mode='max')
mcp_save_ens = ModelCheckpoint(filepath='mdl_wts_en.h5', save_best_only=True, monitor='val_accuracy', mode='max')
mcp_save_ens1 = ModelCheckpoint(filepath='mdl_wts_en1.h5', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', patience=5, verbose=1, min_delta=1e-4, mode='min')

In [ ]:
train_df = pd.read_csv("../input/hackerearth-deep-learning-challenge-holidayseason/dataset/train.csv")

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
TRAINING_DIR = "../input/hackerearth-deep-learning-challenge-holidayseason/dataset/train"
train_datagen = ImageDataGenerator(
    rescale=1/255,
    rotation_range=40,
    width_shift_range=.2,
    height_shift_range=.2,
    shear_range=.2,
    validation_split=0.05,
    #zoom_range=.2,

    horizontal_flip=True,
    #fill_mode='nearest'
)
train_generator = train_datagen.flow_from_dataframe(dataframe=train_df,
    directory = TRAINING_DIR,
    x_col="Image",
    y_col="Class",
    batch_size = 64,
    subset = 'training',
    class_mode='categorical',
    shuffle = True,
    target_size=(150,150)
)

validation_generator = train_datagen.flow_from_dataframe(dataframe=train_df,
    directory = TRAINING_DIR,
    x_col="Image",
    y_col="Class",
    shuffle = True,
    batch_size = 64,
    subset = 'validation',
    class_mode='categorical',
    target_size=(150,150)
)   

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [ ]:
pre_model = InceptionV3( input_shape = (150,150,3), include_top = False )

In [ ]:
for layer in pre_model.layers[:40]:
  layer.trainable = False

In [ ]:
pre_model.summary()

In [ ]:
last_layer = pre_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

In [ ]:
from tensorflow.keras.optimizers import RMSprop

from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras import Model
from os import getcwd
from keras.layers import Dense, Activation

x = layers.Flatten()(last_output)

x = layers.Dropout(0.3)(x)

x = layers.Dense(1024, activation = "relu")(x)

x = layers.Dropout(0.5)(x)

x = layers.Dense(6, activation = "softmax")(x)

model_new = Model(pre_model.input, x)

model_new.compile(optimizer = RMSprop(lr=0.0001), 
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy']
             )

In [ ]:
history_xc = model_new.fit(
      train_generator,  
      callbacks = [ mcp_save_xc, reduce_lr_loss],
      validation_data = validation_generator,
      epochs = 50,
      verbose = 1,
 )

In [ ]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.inception_resnet_v2 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.preprocessing import image
from sklearn.model_selection import train_test_split
import os
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
name=[]
y_pred=[]
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
for i in os.listdir('../input/hackerearth-deep-learning-challenge-holidayseason/dataset/test/'):
    name.append(i)
    i='../input/hackerearth-deep-learning-challenge-holidayseason/dataset/test/'+i
    img=image.load_img(i,target_size=(150,150,3))
    img=image.img_to_array(img)/255
    pred=model_new.predict(img.reshape(1,150,150,3))
    y_pred.append(labels[np.argmax(pred[0])])
    
data=pd.DataFrame((zip(name,y_pred)),columns=['Image','Class'])
data.head()

In [ ]:
data.to_csv('my_submission.csv',index=False)